In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [116]:
Q_learning_df_1 = pd.read_csv("../Data_Generation/Data_files/Q_moves_scores.csv")
Q_learning_df_2 = pd.read_csv("../Data_Generation/Data_files/Q_moves_scores_test.csv")
bfs_df = pd.read_csv("../Data_Generation/Data_files/bfs_heisenberg_data.csv")

In [117]:
all_Q_learning = pd.concat([Q_learning_df_1, Q_learning_df_2])
all_Q_learning = all_Q_learning.drop("last_matrix", axis=1)
all_Q_learning = all_Q_learning.drop_duplicates()

In [118]:
combined_df = pd.merge(all_Q_learning, bfs_df, left_on=["val1", "val2", "val3"], right_on=["val1", "val2", "val3"], how="inner")

In [119]:
print(combined_df.shape)
combined_df.drop_duplicates().shape

(1046, 8)


(1046, 8)

In [121]:
# These Q learning datasets learn a sequence of moves that gets us to the origin 
# as fast as BFS about 97% of the time. 
sum(combined_df['num_moves_Q_learning_needs']==combined_df['num_steps'])/combined_df.shape[0]

0.9780114722753346

In [123]:
set(combined_df['first_move_by_Q_learning'])

{0, 1, 2, 3}

In [91]:
# These Q learning datasets learn a sequence of moves that gets us to the origin 
# as fast as BFS about 35% of the time. 
sum(combined_df['num_moves_Q_learning_needs']==combined_df['num_steps'])/combined_df.shape[0]

0.34565243391938766

In [124]:
# Q learning learns within one move of the optimal 90% of the time
sum(combined_df['num_moves_Q_learning_needs']==1+combined_df['num_steps'])/combined_df.shape[0]

0.0

In [125]:
# Q learning is not learning the same path back to the origin, but it is learning a path of equal length. 
# implies that there are probably multiple different paths of the same length back to the origin for 
# a given element in the heisenberg group

# TODO am I encoding this differently? I don't understand why it is so crazy low. 
sum(combined_df['first_move_by_Q_learning']==combined_df['last_matrix'])

14

In [95]:
set(combined_df['last_matrix'])

{0.0, 1.0, 2.0, 3.0}